In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#import spacy
import re
import string
import numpy as np

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

#from sklearn.linear_model import LogisticRegression
#from sklearn.svm import LinearSVC
#from sklearn.svm import SVC

import torch
from transformers import *

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('imdb/IMDB Dataset.csv')
df['Label'] = 0

In [3]:
df.loc[df.sentiment == 'positive','Label'] = 1
df

,review,sentiment,Label
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1
...,...,...,...
49995,I thought this movie did a down right good job...,positive,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,0
49997,I am a Catholic taught in parochial elementary...,negative,0
49998,I'm going to have to disagree with the previou...,negative,0


In [4]:
STOPWORDS = set.union(set(stopwords.words('english')),set(string.punctuation),set(['...','']))
PUNCTUATION = set.union(set(string.punctuation),set(['...','....','']))
PUNCTUATION.remove('.')
WL = WordNetLemmatizer()

In [5]:
#PUNCTUATION

In [6]:
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertForSequenceClassification.from_pretrained(pretrained_weights,
                                                      output_hidden_states=True,
                                                      output_attentions=True)

In [7]:
def clean_txt(text):
    text = text.lower()
    text =  re.sub(r'<\w+ />','',text)
    text  = re.sub(r"[^a-zA-Z0-9.]+", ' ', text)
    text = re.sub(r'\.+', ".", text)
    text = re.sub(r'\d+(th)?',' ',text) #remove all the numbers
    text = re.sub(r'\d+(th)?','number',text) #remove all the numbers
    #text = re.sub(r'...',' ',text) #remove all the numbers
    #text = re.sub(r'.',' [SEP] ',text) #remove all the numbers
    #text = '[CLS] '+text+' [SEP]'
    return text
    
def lemmatize(text): 
    tokens = text.split(' ')
    return ' '.join([WL.lemmatize(word) for word in tokens])


def add_special_tokens(text):
    sentences = text.split('.')
    if sentences[-1] == '':
        sentences = sentences[:-1]
    joined = ' [SEP] '.join(sentences)
    return joined+' [SEP]'    
    
def clean_stopwords(text):
    return tokens

    
df['cleaned'] = df.review.progress_apply(lambda r: clean_txt(r))
df['lemmatized'] = df.cleaned.progress_apply(lambda r: lemmatize(r))
df['cleaned'] = df.lemmatized.progress_apply(lambda r: add_special_tokens(r))

#df['tokenized'] = df.cleaned.progress_apply(lambda r: tokenize_clean(r))
#df['lemmatized'] = df.tokenized.progress_apply(lambda r: lemmatize(r))

progress-bar: 100%|██████████| 50000/50000 [00:00<00:00, 163744.95it/s]


In [8]:
dfsampled = df.sample(1000)

In [46]:
def tokenize_pad(text,max_len=512):
    tokenized = tokenizer.tokenize(text)
    if len(tokenized) > max_len: 
        tokenized = tokenized[:max_len]
    encoded = tokenizer.encode(tokenized,max_length=max_len)
    size = len(encoded)
    if len(encoded) < max_len:
        encoded = encoded+[0]*(max_len-size)
    return encoded
    
dfsampled['encoded'] = dfsampled.cleaned.progress_apply(tokenize_pad)



progress-bar: 100%|██████████| 1000/1000 [00:10<00:00, 97.17it/s]


In [87]:
lst = dfsampled['encoded'][:10].tolist()
torch.tensor(lst)

tensor([[  101,  2023,  5292,  ...,     0,     0,     0],
        [  101,  2203, 27242,  ...,     0,     0,     0],
        [  101,  1045,  3427,  ...,     0,     0,     0],
        ...,
        [  101,  2023,  2003,  ...,     0,     0,     0],
        [  101,  1045,  2031,  ...,     0,     0,     0],
        [  101,  2028,  1997,  ...,     0,     0,     0]])

In [88]:
model.eval()
with torch.no_grad():
    hidden_states = model(torch.tensor(lst))
    output = hidden_states[1][-1][:,0,:]

In [90]:
output.numpy()

array([[-0.60967004,  0.49433842, -0.36431038, ..., -0.35125175,
         0.7825085 , -0.73858076],
       [-0.7300008 ,  0.1914191 , -0.42763206, ..., -0.39265317,
         0.79416597, -0.43778157],
       [-0.94352895,  0.36732632, -0.55836064, ..., -0.343816  ,
         0.8057951 , -0.54308707],
       ...,
       [-0.74842674,  0.3906517 , -0.55189127, ..., -0.3108351 ,
         0.81164694, -0.49274832],
       [-0.8117502 ,  0.31144175, -0.5445946 , ..., -0.21613792,
         0.89696753, -0.38428256],
       [-0.6234377 , -0.02318926, -0.50432396, ..., -0.19832934,
         0.9186022 , -0.64537215]], dtype=float32)